In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Google sheet - Get handle on it

In [3]:
!pip install --upgrade -q gspread

In [4]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open('fieldbook_pages')
worksheet_pages = sh.worksheet(title="pages")
worksheet_contrasted_pages = sh.worksheet(title="contrasted_pages")

# Download raw images

In [21]:
"""
Loop through rows of spreadsheet
Download images
"""
import requests
import os
row = 2
raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_images/'
for row_counter in range(2,254):
    file_num = worksheet_pages.get_values("C"+str(row_counter))[0][0]
    image_url = worksheet_pages.get_values("D"+str(row_counter))[0][0]
    print(file_num, image_url)
    #ANT_FB_1932-003-0016.jpg
    image_file_name = "ANT_FB_" + file_num + ".jpg"
    img_data = requests.get(image_url).content
    image_file_path = os.path.join(raw_images_dir, image_file_name)
    with open(image_file_path, 'wb') as f:
        f.write(img_data)
        worksheet_pages.update_cell(row_counter, 8, image_file_name)

FB TowerCover https://pi.lib.uchicago.edu/1001/org/ochre/c39123df-8f09-4df8-9126-7f14610c8243&load


# Split images

In [314]:
import requests
import os
import cv2
from PIL import Image as im
# split pages
raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_images/'
raw_split_pages_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_split_pages/'

row = 2
for row_counter in range(2,254):
    file_name = worksheet_pages.get_values("H"+str(row_counter))[0][0]
    image_file_path = os.path.join(raw_images_dir,file_name)
    img = cv2.imread(image_file_path)
  
    # Shape of the image
    print(file_name, "Shape:", img.shape)
    img_shape = img.shape

    page1_file_name = file_name[:-4]+"_p1.jpg"
    page1_file_path = os.path.join(raw_split_pages_dir,
                                       page1_file_name)
    page2_file_name = file_name[:-4]+"_p2.jpg"
    page2_file_path = os.path.join(raw_split_pages_dir,
                                   page2_file_name)

    if img_shape[1]>800 or (img_shape[1] > img_shape[0]):
        num_pages = 2
        page1 = img[0:img_shape[0],0:int(img_shape[1]/2)]
        page2 = img[0:img_shape[0],int(img_shape[1]/2):int(img_shape[1])]
        cv2.imwrite(page1_file_path, page1)
        cv2.imwrite(page2_file_path, page2)
    else:
        num_pages = 1      
        cv2.imwrite(page1_file_path, img)
        page2_file_name = ""
        if os.path.exists(page2_file_path):
            print("removing redundant page 2 file", page2_file_path)           
            os.remove(page2_file_path)

    # update the sheet as a range to prevent Google sheets being overloaded with
    # too many writes
    print(page1_file_name,
      "",
      page2_file_name,
      num_pages,
      str(img.shape[0]),
      str(img.shape[1]))

    worksheet_pages.update('I' +
                       str(row_counter) +
                       ':O' +
                       str(row_counter),
                       [[page1_file_name,
                         "",
                         page2_file_name,
                        "",
                        num_pages,
                        str(img.shape[0]),
                        str(img.shape[1])]])     

ANT_FB_1932-002-0000.jpg Shape: (756, 1050, 3)
ANT_FB_1932-002-0000_p1.jpg  ANT_FB_1932-002-0000_p2.jpg 2 756 1050


In [ ]:
import sys
import cv2
from google.colab.patches import cv2_imshow
#@title Check image split { run: "auto", vertical-output: true }
sheet_row = 2 #@param {type:"integer"}
if sheet_row < 2 or sheet_row > 253:
   sys.exit()
num_pages_init = int(worksheet_pages.get_values("M"+str(sheet_row))[0][0])
if num_pages_init == 1:
   print("This row has", num_pages_init, "page. Does it look correct? Should it be split?")
if num_pages_init == 2:  
   print("This row has", num_pages_init, "pages. Do they both look correct? Should they not be split?")
page1_file_name = worksheet_pages.get_values("I"+str(sheet_row))[0][0]
if len(page1_file_name.strip()) > 0:
    page1_file_path = os.path.join(raw_split_pages_dir,
                                   page1_file_name)
    page1 = cv2.imread(page1_file_path)
    print(page1_file_name, page1_file_path)
    cv2_imshow(page1)
try:    
    page2_file_name = worksheet_pages.get_values("K"+str(sheet_row))[0][0]
    if len(page2_file_name.strip()) > 0:
        page2_file_path = os.path.join(raw_split_pages_dir,
                                       page2_file_name)
        page2 = cv2.imread(page2_file_path)
        print(page2_file_name)
        cv2_imshow(page2)
except:
    print("no page 2")          

# Contrast images
Save a black and white version of each image

In [301]:
def convert_image_to_black_and_white(source_image_path, destination_image_path, threshold = 208):
    raw_img = cv2.imread(source_image_path)
    gray_image = cv2.cvtColor(raw_img, cv2.COLOR_BGR2GRAY)
    #https://pyimagesearch.com/2021/04/28/opencv-thresholding-cv2-threshold/
    #blurred_gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0) -- didn't really work here
    (thresh, binary) = cv2.threshold(gray_image, threshold, 255, cv2.THRESH_BINARY)
    cv2.imwrite(destination_image_path, binary)

In [ ]:
# convert gray images to black and white
import cv2 
from google.colab.patches import cv2_imshow

import os

raw_split_pages_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_split_pages'
contrast_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/contrasted_split_pages' 
for raw_image_file_name in os.listdir(raw_split_pages_dir):
    if raw_image_file_name.endswith(".jpg"):
        print(raw_image_file_name)
        raw_img = cv2.imread(os.path.join(raw_split_pages_dir, raw_image_file_name))
        gray_image = cv2.cvtColor(raw_img, cv2.COLOR_BGR2GRAY)
        #https://pyimagesearch.com/2021/04/28/opencv-thresholding-cv2-threshold/
        #blurred_gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0) -- didn't really work here
        (thresh, binary) = cv2.threshold(gray_image, 208, 255, cv2.THRESH_BINARY)

        cv2.imwrite(os.path.join(contrast_images_dir,raw_image_file_name), binary)

In [30]:
import sys
import cv2
from google.colab.patches import cv2_imshow
#@title Check image conversion to black and white { run: "auto", vertical-output: true }
sheet_row = 3 #@param {type:"integer"}
threshold = 205 #@param {type:"integer"}
if sheet_row < 2 or sheet_row > 253:
   sys.exit()
num_pages_init = int(worksheet_pages.get_values("M"+str(sheet_row))[0][0])
if num_pages_init == 1:
   print("This row has", num_pages_init, "page. Does it look correct? Should it be split?")
if num_pages_init == 2:  
   print("This row has", num_pages_init, "pages. Do they both look correct? Should they not be split?")
page1_file_name = worksheet_pages.get_values("I"+str(sheet_row))[0][0]
if len(page1_file_name.strip()) > 0:
    page1_file_path = os.path.join(raw_split_pages_dir,
                                   page1_file_name)
    page1bw_file_path = os.path.join(contrast_images_dir,
                                   page1_file_name)
    page1 = cv2.imread(page1_file_path)

    convert_image_to_black_and_white(page1_file_path, page1bw_file_path, threshold)
    page1bw = cv2.imread(page1bw_file_path)
    print(page1_file_name, page1_file_path, page1bw_file_path)
    cv2_imshow(page1bw)
    cv2_imshow(page1)

try:    
    page2_file_name = worksheet_pages.get_values("K"+str(sheet_row))[0][0]
    if len(page2_file_name.strip()) > 0:
        page2_file_path = os.path.join(raw_split_pages_dir,
                                       page2_file_name)
        page2bw_file_path = os.path.join(contrast_images_dir,
                                         page2_file_name)
        page2 = cv2.imread(page2_file_path)
        convert_image_to_black_and_white(page2_file_path, page2bw_file_path, threshold)        
        page2bw = cv2.imread(page2bw_file_path)
        print(page2_file_name, page2_file_path, page2bw_file_path)
        cv2_imshow(page2bw)
        cv2_imshow(page2)        
except:
    print("no page 2") 

NameError: ignored

In [26]:
import ipywidgets as widgets
import cv2
from google.colab.patches import cv2_imshow
from ipywidgets import interact, interactive, fixed
from ipywidgets import TwoByTwoLayout
from ipywidgets import GridspecLayout
from IPython.display import display
import os
raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_images/'
raw_split_pages_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_split_pages/'
# https://colab.research.google.com/github/jupyter-widgets/ipywidgets/blob/master/docs/source/examples/Index.ipynb#scrollTo=1Ubw1AeM1435

grid = GridspecLayout(1, 8)

lbl_row = widgets.Label("Row, Page:")
lbl_transcribed = widgets.Label("Transcribed text:")
ta_transcribed = widgets.Textarea(value='-', rows=40, disabled=False)

btn_save = widgets.Button(description="Save edits")

it_row = widgets.IntText(value=2, disabled=False)
btn_load_p1 = widgets.Button(description="Load page 1")
btn_down_row = widgets.Button(description="Down row")
btn_up_row = widgets.Button(description="Up row")

it_page = widgets.IntText(value=1, disabled=False)
btn_load_p2 = widgets.Button(description="Load page 2")

output = widgets.Output()

grid[0, 0] = lbl_row
grid[0, 1] = it_row
grid[0, 2] = it_page
#grid[1,0] = lbl_transcribed
grid[0, 3] = btn_up_row 
grid[0, 4] = btn_down_row
grid[0, 5] = btn_load_p1
grid[0, 6] = btn_load_p2
grid[0, 7] = btn_save

grid2 = GridspecLayout(1, 3)
grid2[0, :1] = output
grid2[0, 2] = ta_transcribed
display(grid,grid2)

def load_row_page(sheet_row, page_num):
    output.clear_output()
    if page_num == 1:
        page_col = "I"
        transcribed_orig = "Q"
        transcribed_contrast = "R"
    else:
        if page_num == 2:
            page_col = "K"
            transcribed_orig = "S"
            transcribed_contrast = "T"

    page_file_name = worksheet_pages.get_values(page_col + str(sheet_row))[0][0]
    num_pages = worksheet_pages.get_values("M" + str(sheet_row))[0][0]
    if len(page_file_name.strip()) > 0:
        page_file_path = os.path.join(raw_split_pages_dir,
                                      page_file_name)
        page = cv2.imread(page_file_path)
        with output:        
            print(page_file_name, "page", page_num, "of", num_pages)
            cv2_imshow(page)
    transcribed_orig_text = worksheet_pages.get_values(transcribed_orig + str(sheet_row))[0][0]
    try:
        transcribed_contrast_text = worksheet_pages.get_values(transcribed_contrast + str(sheet_row))[0][0]
    except:
        transcribed_contrast_text = ""      
    if len(transcribed_orig_text) > len(transcribed_contrast_text):
        transcribed_text = transcribed_orig_text
    else:
        transcribed_text = transcribed_contrast_text
    ta_transcribed.value = transcribed_text

def btn_save_clicked(b):
    page_num = it_page.value
    if page_num == 1:
        edit_save_col = "U"
    else:
        if page_num == 2:
            edit_save_col = "V"

    worksheet_pages.update(edit_save_col +
                       str(it_row.value) +
                       ':' + edit_save_col +
                       str(it_row.value),
                       [[ta_transcribed.value]]) 
    


btn_save.on_click(btn_save_clicked)

def btn_load_p1_clicked(b):
    it_page.value = 1  
    load_row_page(it_row.value, 1)  

def btn_load_p2_clicked(b):
    it_page.value = 2  
    load_row_page(it_row.value, 2)

btn_load_p1.on_click(btn_load_p1_clicked)
btn_load_p2.on_click(btn_load_p2_clicked)

def btn_down_row_clicked(b):
    it_row.value = it_row.value + 1
    it_page.value = 1
    load_row_page(it_row.value, 1)
btn_down_row.on_click(btn_down_row_clicked)

def btn_up_row_clicked(b):
    it_row.value = it_row.value - 1
    it_page.value = 1
    load_row_page(it_row.value, 1)
btn_up_row.on_click(btn_up_row_clicked)

load_row_page(it_row.value, 1)


GridspecLayout(children=(Label(value='Row, Page:', layout=Layout(grid_area='widget001')), IntText(value=2, lay…

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Textarea(value='-', layout=Layout(grid_…

In [14]:
print(t.value)
t.value = "g"

Hello World!ssss
